In [6]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib.patches import Rectangle
from dotenv import load_dotenv

In [16]:
load_dotenv()

def sliding_window(input_list, window_size=3, stride=1):
    result = []
    for i in range(0, len(input_list) - window_size + 1, stride):
        result.append(input_list[i:i + window_size])
    return result

In [ ]:
#Visualize 2.5D slice (-s, s, +s)
im_paths = [os.getenv(f"EXAMPLE_DATA_PATH{i}") for i in range(1, 4)]
grouped_paths = sliding_window(im_paths)
IMG_SIZE = [512,512]

def load_img(path, size=IMG_SIZE):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    shape0 = np.array(img.shape[:2])
    resize = np.array(size)
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        img = np.pad(img, [pady, padx])
        img = img.reshape(tuple(*resize))

    return img


def show_img(img, mask=None):
    plt.imshow(img, cmap='bone')
    #plt.axis('off')
    
def load_imgs(img_paths, size=IMG_SIZE):
    imgs = np.zeros((*size, len(img_paths)), dtype=np.uint16)
    for i, img_path in enumerate(img_paths):
        img = load_img(img_path, size=size)
        imgs[..., i]+=img
    return imgs

for group in grouped_paths:
    plt.figure(figsize=(20, 10))

    plt.subplot(1, 2, 1)
    img = load_img(group[0]).astype('float32')
    img/=img.max()
    plt.imshow(img)
    #plt.axis('off')


    plt.subplot(1, 2, 2)
    imgs = load_imgs(group).astype('float32')
    imgs/=imgs.max(axis=(0,1))
    plt.imshow(imgs)
    #plt.axis('off')

    #plt.tight_layout()
    plt.show()